# Generate the data for the ISDM-24 track

In [18]:
%pip install -q pandas openpyxl jinja2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Print formatted dates in spanish locale
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

In [2]:
# Read the dataframe from the .xlsx file given by Easychair
import pandas as pd
df = pd.read_excel('../info/ISDM2024.xlsx',index_col=0,header=0)
df

,Authors,Title,Tipo,Abstract,Keywords
#,,,,,
2,"Javier Cámara, Javier Troya, Lola Burgueño and...",On the Assessment of Generative AI in Modeling...,relevante,Most experts agree that Large Language Models ...,"Large Language Models, ChatGPT, Software model..."
3,"Meriem Ben Chaaben, Lola Burgueño and Houari S...",Towards using Few-Shot Prompt Learning for Aut...,relevante,We propose a simple yet a novel approach to im...,"Language models, few-shot learning, prompt lea..."
4,"Meysam Karimi, Shekoufeh Kolahdouz Rahimi and ...",Ant-colony optimization for automating test mo...,relevante,"In model transformation (MT) testing, test dat...","Model-Driven Engineering, Model Transformation..."
12,"Cesar Gonzalez Mora, Cristina Barros, Irene Ga...",Improving Open Data Web API Documentation thro...,relevante,Widely adoption of Information Technologies ha...,"Web API, OpenAPI documentation, Natural Langua..."
24,Pedro Juan Molina Moreno and Javier Centeno Vega,Orca: Nocode Graphical Editor for Container Or...,corto,Traditional Container Orchestration tools like...,"MDE, DSL, nocode, containers, graphical editor..."
57,"Rubén Salado-Cid, Antonio Vallecillo, Kamran M...",SWEL: A Domain-Specific Language for Modeling ...,relevante,Data-intensive applications aim at discovering...,"Model-driven engineering, Domain-specific mode..."
81,"Guillermo Fuentes, Francisco Ruiz and Angelica...",Modelado dirigido por las preocupaciones de lo...,herramienta,Un punto de vista contiene un conjunto de acue...,"Arquitectura Empresarial, Archi, ArchiMate, Pu..."
91,"Abel Gómez, Iván Alfonso, Silvia Doñate, Jordi...",Role-based access control in wastewater treatm...,completo,Wastewater treatment plants (WWTPs) are critic...,"Model-driven engineering (MDE), Role-based acc..."
94,"José Ángel Barriga Corchero, Pedro José Clemen...","Design, code generation and simulation of IoT ...",relevante,The Internet of Things (IoT) is rapidly spread...,"IIoT systems, IoT simulation, IoT mobility, Mo..."


In [12]:
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Session:
    name: str
    start: datetime
    end: datetime
    location: str
    chair: str

isdm1 = Session('ISDM-1',
                datetime(2024,7,18,9,15),
                datetime(2024,7,18,10,15),
                'Aula A.3.1',
                'Diego Sevilla')

isdm2 = Session('ISDM-2',
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,17,30),
                'Aula A.3.1',
                'Pedro J. Clemente')

isdm3 = Session('ISDM-3',
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,19,00),
                'Aula A.3.1',
                'Diego Sevilla y Pedro J. Clemente')

sessions = {s.name : s for s in [isdm1, isdm2, isdm3]}

session_map = {isdm1.name: [91, 94, 24],
               isdm2.name: [3, 2, 4, 121, 124],
               isdm3.name: [12, 57, 81]}

assert set(session_map.keys()) == set(sessions.keys())
# No article left behind
assert set(df.index) == set().union(*session_map.values())

In [17]:
#print(isdm1.start.strftime('%A, %d de %B de %Y, %H:%M'))

jueves, 18 de julio de 2024, 09:15


In [23]:
# Produce the slide index from the slideindex template
from jinja2 import Template
template = Template(open('templates/sessionindex.tex.j2').read())
with open('../pr/sessionindex.tex', 'w') as f:
    f.write(template.render(sessions=sessions))

In [37]:
template = Template(open('templates/sessionslide.tex.j2').read())

for session, slides in session_map.items():
    with open(f'../pr/{session}.tex', 'w') as f:
        f.write(template.render(session=sessions[session],
                                session_map=session_map,
                                startdate = sessions[session].start.strftime('%A, %d de %B de %Y, %H:%M'),
                                df=df))